In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


hf_token = os.environ.get('hf_token')

# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [6]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Given a CSV dataset named 'ue-lte-network-traffic-stats.csv' with columns UE1: web-rtc	UE1: sipp	UE1: web-server	UE2: web-rtc	UE2: sipp	UE2: web-server	UE3: web-rtc	UE3: sipp	UE3: web-server	UE1-Jitter	UE2-Jitter	UE3-Jitter	UE1-CQI	UE2-CQI	UE3-CQI, produce Python code to perform Traffic analysis by calculating the total traffic for each UE and generate plot of the total traffic"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Given a CSV dataset named 'ue-lte-network-traffic-stats.csv' with columns UE1: web-rtc	UE1: sipp	UE1: web-server	UE2: web-rtc	UE2: sipp	UE2: web-server	UE3: web-rtc	UE3: sipp	UE3: web-server	UE1-Jitter	UE2-Jitter	UE3-Jitter	UE1-CQI	UE2-CQI	UE3-CQI, produce Python code to perform Traffic analysis by calculating the total traffic for each UE and generate plot of the total traffic [/INST]  Sure, here's an example of how you could perform traffic analysis on the given CSV dataset and generate plots of the total traffic for each UE:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV dataset
df = pd.read_csv('ue-lte-network-traffic-stats.csv')

# Calculate the total traffic for each UE
df['Total Traffic'] = df['UE1: web-rtc'] + df['UE1: sipp'] + df['UE1: web-server'] + \
                      df['UE2: web-rtc'] + df['UE2: sipp'] + df['UE2: web-server'] + \
                      df['UE3: web-rtc'] + df['UE3: sipp'] + df['UE3: web-server'

In [8]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Given a CSV dataset named 'ue-lte-network-traffic-stats.csv' with columns UE1: web-rtc	UE1: sipp	UE1: web-server	UE2: web-rtc	UE2: sipp	UE2: web-server	UE3: web-rtc	UE3: sipp	UE3: web-server	UE1-Jitter	UE2-Jitter	UE3-Jitter	UE1-CQI	UE2-CQI	UE3-CQI, produce Python code to perform Traffic analysis by calculating the total traffic for each UE and plot the traffic"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Given a CSV dataset named 'ue-lte-network-traffic-stats.csv' with columns UE1: web-rtc	UE1: sipp	UE1: web-server	UE2: web-rtc	UE2: sipp	UE2: web-server	UE3: web-rtc	UE3: sipp	UE3: web-server	UE1-Jitter	UE2-Jitter	UE3-Jitter	UE1-CQI	UE2-CQI	UE3-CQI, produce Python code to perform Traffic analysis by calculating the total traffic for each UE and plot the traffic [/INST]  Sure! Here's the Python code to perform traffic analysis on the given CSV dataset and plot the traffic for each UE:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV dataset
df = pd.read_csv('ue-lte-network-traffic-stats.csv')

# Calculate the total traffic for each UE
UE1_traffic = df['UE1: web-rtc'] + df['UE1: sipp'] + df['UE1: web-server']
UE2_traffic = df['UE2: web-rtc'] + df['UE2: sipp'] + df['UE2: web-server']
UE3_traffic = df['UE3: web-rtc'] + df['UE3: sipp'] + df['UE3: web-server']

# Calculate the jitter for each UE
UE1_jitter = df['UE1-Jitter']
UE2_jitter

In [ ]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_multi_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["choices"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer: {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    match = re.search(r'(Answer:|Correct answer:)\s*((?:[A-D]\)\s*)?[\w\s()\-/]+)', generated_answer)
    if match:
        parsed_answer = match.group(2).strip()
        
        # Exclude the "Explanation" part from the parsed answer if present
        explanation_start = parsed_answer.find("\nExplanation")
        if explanation_start != -1:
            parsed_answer = parsed_answer[:explanation_start].strip()
        
        print(f"Parsed_answer: {parsed_answer}")
        generated_answers.append(parsed_answer)
    else:
        print("No answer detected.")
        generated_answers.append("N/A")
    
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

In [4]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Generate a Python script that analyzes packet captures from N3 and N6 network interfaces. Use Wireshark to process these captures into PDML format. Parse the PDMLs to determine packet delay based on timestamps and classify packets by source and destination IPs."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2048)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate a Python script that analyzes packet captures from N3 and N6 network interfaces. Use Wireshark to process these captures into PDML format. Parse the PDMLs to determine packet delay based on timestamps and classify packets by source and destination IPs. [/INST]  Sure! Here is a Python script that analyzes packet captures from N3 and N6 network interfaces, uses Wireshark to process the captures into PDML format, and then parses the PDMLs to determine packet delay based on timestamps and classifies packets by source and destination IPs:
```
import subprocess
import re
import pandas as pd

# Set the capture files
n3_capture = 'n3_capture.pcap'
n6_capture = 'n6_capture.pcap'

# Run Wireshark to convert the captures to PDML format
subprocess.run(['wireshark', '-i', 'n3', '-o', 'n3_pdml.txt', n3_capture])
subprocess.run(['wireshark', '-i', 'n6', '-o', 'n6_pdml.txt', n6_capture])

# Read the PDML files
n3_pdml = open('n3_pdml.txt', 'r')
n6_pdml = open('n6_pdml.txt', 'r')

# 

In [3]:
import json
input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_SPEC_QA.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]What are the key objectives and goals of 5G network specifications, and how do they differ from previous generations of wireless networks?[/INST]  The key objectives and goals of 5G network specifications are to provide a high-speed, low-latency, and highly reliable wireless communication infrastructure that supports a wide range of new and innovative services and applications. Some of the key objectives and goals of 5G networks include:

1. Higher speeds: 5G networks are designed to provide much higher speeds than previous generations of wireless networks. The International Telecommunication Union (ITU) has set a goal of providing speeds of up to 20 Gbps in 5G networks.
2. Lower latency: 5G networks are designed to provide much lower latency than previous generations of wireless networks. The ITU has set a goal of providing latency of less than 1 ms in 5G networks.
3. Increased capacity: 5G networks are designed to provide much higher capacity than previous generations o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Explain the architectural components of a typical 5G network, including the roles of gNodeBs, Core Network, and Edge Computing.[/INST]  A typical 5G network consists of several architectural components that work together to provide high-speed, low-latency, and highly reliable wireless connectivity. The key components of a 5G network include:

1. gNodeBs (Next-generation NodeBs): These are the base stations that provide wireless connectivity to mobile devices. gNodeBs are the equivalent of the eNodeBs in 4G networks, but they are designed to support the higher frequency bands and the more complex radio protocols required for 5G.
2. Core Network: This is the central part of the network that manages the communication between gNodeBs and other network elements. The core network includes the following components:

a. MME (Mobility Management Entity): This entity manages the mobility of mobile devices, including the tracking of device locations and the handling of handovers bet

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]What is the significance of mmWave frequencies in 5G, and how do they impact network coverage and capacity?[/INST]  Millimeter wave (mmWave) frequencies are a key component of 5G wireless networks, offering significant improvements in network coverage and capacity compared to previous generations of wireless technology. Here are some key aspects of mmWave frequencies in 5G and their impact on network coverage and capacity:

1. High frequency bands: mmWave frequencies are typically defined as frequencies above 24 GHz, which are much higher than the frequencies used in 4G networks (typically below 6 GHz). These high frequencies offer several advantages, including:

a. Larger bandwidth: mmWave frequencies have much larger bandwidths than lower frequency bands, which allows for more data to be transmitted in the same amount of time.

b. Lower latency: The shorter wavelengths of mmWave frequencies result in lower latency, which is essential for applications that require real-t

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Describe the concept of network slicing in 5G, and provide examples of how it benefits different industries and applications.[/INST]  Network slicing is a key feature of 5G networks that allows multiple independent network slices to coexist on the same physical infrastructure. Each slice can be customized to meet the specific needs of a particular application or industry, such as IoT, autonomous vehicles, or smart cities. This concept enables the creation of dedicated networks for specific use cases, providing better performance, security, and resource allocation.

Here are some examples of how network slicing benefits different industries and applications:

1. IoT (Internet of Things): Network slicing allows for the creation of a dedicated network for IoT devices, ensuring low latency and high reliability for applications such as smart home systems, industrial automation, and smart cities.
2. Autonomous vehicles: A network slice can be dedicated to autonomous vehicles, p

`Network config QA`: Pass

In [ ]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_multiple_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"{question}\nChoose the correct answer:\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
        # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

In [11]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/bgp_test_multiple_choice_v2.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"{question}\nChoose the correct answer:\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
        # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Generated Answer: When analyzing BGP route propagation, which attribute provides information about the path that the BGP update has traversed?
Choose the correct answer:
a) AS-Path
b) MED (Multi-Exit Discriminator)
c) Local Preference
d) Next Hop

Answer: a) AS-Path

Explanation:

The AS-Path attribute in BGP updates contains a list of autonomous systems (ASes) that the update has traversed. This attribute provides information about the path that the BGP update has taken from the originating AS to the receiving
Parsed_answer: a
----------------1-----------------
Generated Answer: Which of the following is a primary purpose of BGP?
Choose the correct answer:
a) Establishing communication between devices within a LAN
b) Exchanging routing information between autonomous systems
c) Optimizing local network traffic
d) Assigning IP addresses to devices

Answer: b) Exchanging routing information between autonomous systems

Explanation:

BGP (Border Gateway Protocol) is a routing protocol used

In [3]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_fill_the_blank.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
filled_answers = []

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST]{question} \n Fill the blank: \n [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    # print(f"Output: {generated_answer}")
    
    generated_answer = re.search(r'\[/INST\]\s+(.*)', generated_answer, re.DOTALL).group(1).strip()
    if "Sure" in generated_answer:
        generated_answer = generated_answer.split('\n', 1)[1]  # Skip the first line
    print(f"Generated Answer: {generated_answer}")
    generated_answers.append(generated_answer)
    
    # Find the position of the placeholder in the first string
    placeholder_position = question.find("________")

    # Extract the content filled in the placeholder
    filled_content = generated_answer[placeholder_position:placeholder_position + len("autonomous systems (ASes)")]
    filled_answers.append(filled_content)
    n_question += 1
    print(f"Filled content: {filled_content}")
    print(f"---------------------{n_question}-----------------------------")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
BGP (Border Gateway Protocol) is a standardized exterior gateway protocol used to exchange routing and reachability information between Autonomous Systems (AS) on the Internet. It enables the selection of paths and routes for data to flow efficiently between networks.
Filled content:  Autonomous Systems (AS) 
---------------------1-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
In BGP, the AS-Path attribute represents the sequence of autonomous systems (ASes) that a route has traversed.

So, the correct answer is:

"sequence of autonomous systems (ASes)"
Filled content:  sequence of autonomous s
---------------------2-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: BGP route selection ______________ involves manipulating the BGP attributes to control the selection of routes.

The correct answer is: BGP route selection ______________ involves manipulating the BGP attributes to control the selection of routes.

Explanation:

BGP (Border Gateway Protocol) is a routing protocol used to exchange routing information among autonomous systems (AS) on the internet. BGP route
Filled content: selection ______________ 
---------------------33-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
BGP peering is used for establishing BGP session connections and exchanging routing information.
Filled content:  peering is used for esta
---------------------34-----------------------------
Generated Answer: The phenomenon where the Multi-Exit Discriminator (MED) value of a route frequently changes is referred to as "MED oscillations."

Therefore, the correct answer is:

MED oscillations.
Filled content:  as "MED oscillations."


---------------------35-----------------------------


: 